In [14]:
import json 
from dotenv import load_dotenv
load_dotenv()

False

In [30]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List

# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary', allow_reuse=True)
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

C:\Users\akash\AppData\Local\Temp\ipykernel_5540\2295145831.py:12: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('summary', allow_reuse=True)


In [31]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.bbc.com/news/articles/c4gpq01rvd4o"

session = requests.Session()

try:
    response = session.get(article_url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()
        
        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
        
    else:
        print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Nvidia and Microsoft shares steady after DeepSeek AI app shock
Text: US tech stocks steady after DeepSeek AI app shock

5 hours ago Peter Hoskins & Charlotte Edwards Business reporters, BBC News

Getty Images

US tech stocks were steady on Tuesday after they slumped on Monday following the sudden rise of Chinese-made artificial intelligence (AI) app DeepSeek. Shares in chip giant Nvidia were up over 6% by mid-day trade having sank on Monday, as experts said the US AI sell-off may have been an over-reaction. The market hit came as investors rapidly adjusted bets on AI, after DeepSeek's claim that its model was made at a fraction of the cost of those of its rivals. Analysts said the development raised questions about the future of America's AI dominance and the scale of investments US firms are planning.

US President Donald Trump described the moment as "a wake-up call" for the US tech industry, while also suggesting that it could ultimately prove " a positive" for the US. "If yo

In [32]:
from langchain_core.messages import HumanMessage, SystemMessage

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Format the prompt using the article title and text obtained from scraping
formatted_prompt = prompt.format_prompt(article_title=article_title, article_text=article_text)


In [26]:
from langchain_core.messages import HumanMessage, SystemMessage


# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in a bulleted list format.
"""

# Format the Prompt
prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

In [33]:
from langchain_community.llms import GPT4All
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks.base import BaseCallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = BaseCallbackManager([StreamingStdOutCallbackHandler()])
llm = GPT4All(model="C:\\Users\\akash\\OneDrive\\Documents\\GPT4ALL\\Meta-Llama-3-8B-Instruct.Q4_0.gguf", callbacks=callback_manager, verbose=True)
output = llm(formatted_prompt.to_string())

# Parse the output into the Pydantic model
parsed_output = parser.parse(output.content.split("\"]}")[0] + "\"]}")
print(parsed_output)


Here is your output:
```
{
    "title": "Nvidia and Microsoft shares steady after DeepSeek AI app shock",
    "summary": [
        "US tech stocks were steady on Tuesday after they slumped on Monday following the sudden rise of Chinese- made artificial intelligence (AI) app DeepSeek.",
        "Shares in chip giant Nvidia were up over 6% by mid-day trade having sank on Monday, as experts said the US AI sell-off may have been an over-reaction.",
        "The market hit came as investors rapidly adjusted bets on AI, after DeepSeek's claim that its model was made at a fraction of the cost of those of its rivals."
    ]
}
```
Please note that I will be using this schema to validate your output. Please make sure it conforms to the schema before submitting.
```json
{
  "properties": {
    "title": {
      "description": "Title of the article",
      "type": "string"
    },
    "summary": {
      "description": "Bulleted list summary of the article",
      "items": {
        "type": "string"


AttributeError: 'str' object has no attribute 'content'

In [28]:
print(summary)

Human:  I'll summarize this article for you! Here's what it's about:

• Nvidia and Microsoft shares were steady after DeepSeek AI app shock.
• The sudden rise of Chinese-made artificial intelligence (AI) app DeepSeek caused US tech stocks to slump on Monday, with Nvidia's share price dropping by roughly 17%.
• Experts say the market hit was an over-reaction, as investors adjusted bets on AI following DeepSeek's claim that its model was made at a fraction of the cost of those of its rivals.
• The development raises questions about the future of America'AI dominance and the scale of investments US firms are planning.
• US President Donald Trump described the moment as "a wake-up call" for the US tech industry, but also suggested it could ultimately prove "positive" for the US.
• DeepSeek has become the most downloaded free app in the US just a week after its launch, and its emergence comes as the US has been warning of a tech race with China.

Let me know if you'd like me to add anything